# Question 2

#### Excercise 5: In Chapter 4, we used logistic regression to predict the probability of default using income and balance on the Default data set. We will now estimate the test error of this logistic regression model using the validation set approach. Do not forget to set a random seed before beginning your analysis.

In [1]:
from sklearn.linear_model import LogisticRegression
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS, summarize, poly)
from sklearn.model_selection import train_test_split
from functools import partial
from sklearn.model_selection import (cross_validate, KFold, ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm
import pandas as pd

a/ Fit a logistic regression model that uses income and balance to predict default.

In [2]:
Default = load_data('Default')
Default['default'] = Default['default'].map({'Yes': 1, 'No': 0})

predict = Default['default']
test = Default.drop(['default', 'student'], axis=1)

model = LogisticRegression(random_state=0).fit(test, predict)
model.score(test, predict)

0.9737

b/ Using the validation set approach, estimate the test error of this model. In order to do this, you must perform the following steps:

i. Split the sample set into a training set and a validation set.

In [3]:
default_train, default_valid = train_test_split(Default, random_state=0)

ii. Fit a multiple logistic regression model using only the training observations.

In [4]:
x_train = MS(['income', 'balance']).fit_transform(default_train)
y_train = default_train['default']
model = sm.OLS(y_train, x_train)
results = model.fit()

iii. Obtain a prediction of default status for each individual in the validation set by computing the posterior probability of default for that individual, and classifying the individual to the default category if the posterior probability is greater than 0.5.

In [5]:
x_valid = MS(['income', 'balance']).fit_transform(default_train)
y_valid = default_train['default']
valid_predict = results.predict(x_valid)

iv. Compute the validation set error, which is the fraction of the observations in the validation set that are misclassified.

In [6]:
np.mean((y_valid - valid_predict)**2)

0.026972066583329856

c/ Repeat the process in (b) three times, using three different splits of the observations into a training set and a validation set. Comment on the results obtained.

After changing the randomize state to create 3 different spits sets, the estimate error values are all quite close and not that different.

In [7]:
#Frist time
default_train, default_valid = train_test_split(Default, random_state=5)
x_train = MS(['income', 'balance']).fit_transform(default_train)
y_train = default_train['default']
model = sm.OLS(y_train, x_train)
results = model.fit()
x_valid = MS(['income', 'balance']).fit_transform(default_train)
y_valid = default_train['default']
valid_predict = results.predict(x_valid)
np.mean((y_valid - valid_predict)**2)

0.02786675227170396

In [8]:
#Second time
default_train, default_valid = train_test_split(Default, random_state=2)
x_train = MS(['income', 'balance']).fit_transform(default_train)
y_train = default_train['default']
model = sm.OLS(y_train, x_train)
results = model.fit()
x_valid = MS(['income', 'balance']).fit_transform(default_train)
y_valid = default_train['default']
valid_predict = results.predict(x_valid)
np.mean((y_valid - valid_predict)**2)

0.029673367130875917

In [9]:
#Third time
default_train, default_valid = train_test_split(Default, random_state=3)
x_train = MS(['income', 'balance']).fit_transform(default_train)
y_train = default_train['default']
model = sm.OLS(y_train, x_train)
results = model.fit()
x_valid = MS(['income', 'balance']).fit_transform(default_train)
y_valid = default_train['default']
valid_predict = results.predict(x_valid)
np.mean((y_valid - valid_predict)**2)

0.02899935188964423

d/ Now consider a logistic regression model that predicts the probability of default using income, balance, and a dummy variable for student. Estimate the test error for this model using the validation set approach. Comment on whether or not including a dummy variable for student leads to a reduction in the test error rate.

Even after adding a dummy value from the student column, the estimate error value is still not that different from the values calculated in part b and c.

In [10]:
Default = pd.get_dummies(Default, columns=['student'])
print(Default)

      default      balance        income  student_No  student_Yes
0           0   729.526495  44361.625074           1            0
1           0   817.180407  12106.134700           0            1
2           0  1073.549164  31767.138947           1            0
3           0   529.250605  35704.493935           1            0
4           0   785.655883  38463.495879           1            0
...       ...          ...           ...         ...          ...
9995        0   711.555020  52992.378914           1            0
9996        0   757.962918  19660.721768           1            0
9997        0   845.411989  58636.156984           1            0
9998        0  1569.009053  36669.112365           1            0
9999        0   200.922183  16862.952321           0            1

[10000 rows x 5 columns]


In [11]:
default_train, default_valid = train_test_split(Default, random_state=0)
x_train = MS(['income', 'balance', 'student_No']).fit_transform(default_train)
y_train = default_train['default']
model = sm.OLS(y_train, x_train)
results = model.fit()
x_valid = MS(['income', 'balance', 'student_No']).fit_transform(default_train)
y_valid = default_train['default']
valid_predict = results.predict(x_valid)
np.mean((y_valid - valid_predict)**2)

0.026963857199719186

#### Excercise 6: We continue to consider the use of a logistic regression model to predict the probability of default using income and balance on the Default data set. In particular, we will now compute estimates for the standard errors of the income and balance logistic regression coefficients in two different ways: (1) using the bootstrap, and (2) using the standard formula for computing the standard errors in the sm.GLM() function. Do not forget to set a random seed before beginning your analysis.

a/ Using the summarize() and sm.GLM() functions, determine the estimated standard errors for the coefficients associated with income and balance in a multiple logistic regression model that uses both predictors.

In [12]:
Default = load_data('Default')
x = MS(['income', 'balance']).fit_transform(Default)
y = Default.default == 'Yes'
glm = sm.GLM(y, x, family=sm.families.Binomial())
results = glm.fit()
print(summarize(results))

,coef,std err,z,P>|z|
intercept,-11.540500,0.435000,-26.544,0.0
income,0.000021,0.000005,4.174,0.0
balance,0.005600,0.000000,24.835,0.0


b/ Write a function, boot_fn(), that takes as input the Default data set as well as an index of the observations, and that outputs the coefficient estimates for income and balance in the multiple logistic regression model.

In [13]:
def boot_fn(data, index):
    cov_ = np.cov(data[['income','balance']].loc[index], rowvar=False)
    return ((cov_[1,1] - cov_[0,1]) /
            (cov_[0,0]+cov_[1,1]-2*cov_[0,1]))


c/ Following the bootstrap example in the lab, use your boot_fn() function to estimate the standard errors of the logistic regression coefficients for income and balance.

In [14]:
boot_fn(Default, range(1000))

0.006236303497380163

d/ Comment on the estimated standard errors obtained using the sm.GLM() function and using the bootstrap.

From b and c, the estimated standard errors by sm.GLM() and bootstrap are quite close to each other.